In [ ]:
#| default_exp quantize.quantizer

In [ ]:
#| export
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.ao.quantization import get_default_qconfig_mapping
import torch.ao.quantization.quantize_fx as quantize_fx
from torch.ao.quantization.quantize_fx import convert_fx, prepare_fx

In [ ]:
#| include: false
from nbdev.showdoc import *
from fastai.vision.all import *
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#| export
class Quantizer():
    def __init__(self, backend="x86"):
        self.qconfig = get_default_qconfig_mapping(backend)
    
    def quantize(self, model, calibration_dl):
        x, _ = calibration_dl.valid.one_batch()
        model_prepared = prepare_fx(model.eval(), self.qconfig, x)
        _ = [model_prepared(xb.to('cpu')) for xb, _ in calibration_dl.valid]
            
        return convert_fx(model_prepared)

In [ ]:
show_doc(Quantizer)

---

[source](https://github.com/nathanhubens/fasterai/tree/master/blob/master/fasterai/quantize/quantizer.py#L16){target="_blank" style="float:right; font-size:smaller"}

### Quantizer

>      Quantizer (activation_observer=<class
>                 'torch.ao.quantization.observer.MinMaxObserver'>,
>                 weight_observer=<class
>                 'torch.ao.quantization.observer.MinMaxObserver'>,
>                 activation_qtype=torch.qint8, weight_qtype=torch.quint8,
>                 granularity='tensor')

Initialize self.  See help(type(self)) for accurate signature.